# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
cities_df=pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.42,19.23,60.01,79.0,2.0,5.01,ZA,1593506973
1,Hithadhoo,-0.60,73.08,84.65,64.0,15.0,3.40,MV,1593506703
2,Kapaa,22.08,-159.32,78.80,78.0,90.0,11.41,US,1593506975
3,Rikitea,-23.12,-134.97,69.06,58.0,84.0,18.21,PF,1593506976
4,Katsuura,35.13,140.30,78.80,94.0,75.0,16.11,JP,1593506802


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations
locations=cities_df[["Lat","Lng"]]

# create heatmap
fig = gmaps.figure(center=(0,0),zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(
    locations,
    weights=cities_df["Humidity"],
    dissipating=False,
    max_intensity=cities_df["Humidity"].max(),
    point_radius=1
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
cities_df=cities_df[
    (cities_df["Max Temp"]<80) & 
    (cities_df["Max Temp"]>70) & 
    (cities_df["Wind Speed"]<10) & 
    (cities_df["Cloudiness"]==0)
]
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
243,Yulara,-25.24,130.99,78.80,15.0,0.0,4.70,AU,1593507160
386,Marrakesh,31.63,-8.01,77.00,65.0,0.0,4.70,MA,1593507269
400,Bindura,-17.30,31.33,73.80,30.0,0.0,5.55,ZW,1593507278
420,Shakawe,-18.37,21.85,71.89,20.0,0.0,8.25,BW,1593507295
554,Itaueira,-7.60,-43.03,70.61,70.0,0.0,8.75,BR,1593507400
569,Grand Gaube,-20.01,57.66,77.00,67.0,0.0,5.99,MU,1593507411


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df=pd.DataFrame()

params={
    "radius":5000,
    "types":"lodging",
    "keyword":"hotel",
    "key":g_key
}

for index,row in cities_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
        hotel_df.loc[index,"City"]=row["City"]
        hotel_df.loc[index,"Country"]=row["Country"]
        hotel_df.loc[index,"Lat"]=response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index,"Lng"]=response["results"][0]["geometry"]["location"]["lng"]
    except (KeyError,IndexError):
        continue
hotel_df

,Hotel Name,City,Country,Lat,Lng
243,Outback Pioneer Hotel & Lodge - Ayers Rock Resort,Yulara,AU,-25.243161,130.989527
386,Sofitel Marrakech Lounge & Spa,Marrakesh,MA,31.621689,-8.002691
400,Rutati Guest House,Bindura,ZW,-17.303952,31.335484
420,Athoma Guest House,Shakawe,BW,-18.368630,21.849130
554,Hotel Pousada Luz,Itaueira,BR,-7.598781,-43.030338
569,Paradise Cove Boutique Hotel,Grand Gaube,MU,-19.987890,57.632652


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))